# Retrieving NZ Demographics Information

Author: Reean Liao <br/>
Date: June 2019

## Table of Contents

* [Population from Wikipedia](#Population-from-Wikipedia)
* [Income from StatsNZ](#Income-from-StatsNZ)
<br/>
<br/>	  
* [Reference](#Reference)
<br/>
<br/>    
* [Appendix](#Appendix)

In [1]:
import pandas as pd
import numpy as np
from pandas import DataFrame,Series
import re # regex (regular expression) module
import requests,lxml                 
from bs4 import BeautifulSoup  
import pprint # PrettyPrint for pretty printing json
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style="darkgrid")
plt.style.use('ggplot')
%matplotlib inline

## Introduction
[Back to top](#Retrieving-NZ-Demographics-Information)

Retrieving New Zealand's demographic/census data is a tricky task. In general, StatsNZ is the central source of all census related data, but there's no easy way of querying it. Until a better API$^{①}$ comes along, the best way to obtain such information is to search for it in StatsNZ's website (https://www.stats.govt.nz/) and download what you need as CSV.

This document shows examples of an alternative solution - obtaining NZ demographics information from web scraping.

①:_StatsNZ had an API initiative in 2017 to 2018; the Appendix shows how it would have worked, but this API is no longer available._

## Population from Wikipedia
[Back to top](#Retrieving-NZ-Demographics-Information)

Suppose you manage to find the population by regions information from a Wikipedia web page (Regions of New Zealand, n.d.). You can retrieve the information using the method below. Note that the snapshot data here is a June 2018 provisional estimate given by StatsNZ.

In [2]:
url = "https://en.wikipedia.org/wiki/Regions_of_New_Zealand"
response = requests.get(url)
print response
scraping_wiki = BeautifulSoup(response.content, "lxml")

<Response [200]>


The request was sent to the Wikipedia webpage and a successful response is received. Since our target table is a "Wikitable" class attribute, we can locate the target table using the command below.

In [3]:
#Retrieve all wikitables
wiki_table=scraping_wiki.findAll("table", attrs={"class" : "wikitable"})

#Pandas built in function to read html
table_list = pd.read_html(str(wiki_table))
type(table_list)

list

The Wikipedia web page contains two tables, but we will only retrieve the first table on the list. The table containing population is presented below.

In [26]:
df_all=table_list[0]
df_all

,\n,Region name(name in M\u0101ori if different)\n,Regional council\n,Chair\n,Council seat\n,Island\n,Area\xa0(km\xb2)[19]\n,Population[20]\n,Density\n(pop./km\xb2)\n\n,ISO 3166-2 Code\n
0,\n,Region name(name in M\u0101ori if different)\n,Regional council\n,Chair\n,Council seat\n,Island\n,Area\xa0(km\xb2)[19]\n,Population[20]\n,Density\n(pop./km\xb2)\n\n,ISO 3166-2 Code\n
1,1,Northland Te Tai Tokerau,Northland Regional Council,Bill Shepherd[21],Whangarei,North,12498,"179,100\n",13.71,NZ-NTL\n
2,2,Auckland(1) T\u0101maki-makau-rau,Auckland Council,Phil Goff,Auckland,North,4940,"1,695,900\n",326.78,NZ-AUK\n
3,3,Waikato,Waikato Regional Council,Alan Livingston,Hamilton,North,23900,"468,800\n",18.79,NZ-WKO\n
4,4,Bay of Plenty Te Moana-a-Toi,Bay of Plenty Regional Council,Doug Leeder,Whakatane,North,12071,"305,700\n",24.31,NZ-BOP\n
5,5,Gisborne(1)(2) Te Tai R\u0101whiti,Gisborne District Council,Meng Foon,Gisborne,North,8386,"49,100\n",5.71,NZ-GIS\n
6,6,Hawke's Bay Te Matau-a-M\u0101ui,Hawke's Bay Regional Council,Rex Graham,Napier,North,14137,"165,900\n",11.42,NZ-HKB\n
7,7,Taranaki,Taranaki Regional Council,David MacLeod,Stratford,North,7254,"119,600\n",16.07,NZ-TKI\n
8,8,Manawatu-Wanganui Manawat\u016b-Whanganui,Horizons Regional Council,Bruce Gordon,Palmerston North,North,22221,"234,500\n",10.55,NZ-MWT\n
9,9,Wellington Te Whanga-nui-a-Tara,Greater Wellington Regional Council,Chris Laidlaw[22],Wellington,North,8049,"521,500\n",62.73,NZ-WGN\n


For now, let's only keep the region and population field. We'll use the `"Regional council"` field since the `"Region Name"` field is mixed with region Maori names - harder to clean.

In [28]:
#Select only Region and Population
df_pop=df_all[['Regional council\\n','Population[20]\\n']] #used df_pop.columns.values to find out column names

#Rename columns
df_pop.columns=['Region','Population']
df_pop 

,Region,Population
0,Regional council\n,Population[20]\n
1,Northland Regional Council,"179,100\n"
2,Auckland Council,"1,695,900\n"
3,Waikato Regional Council,"468,800\n"
4,Bay of Plenty Regional Council,"305,700\n"
5,Gisborne District Council,"49,100\n"
6,Hawke's Bay Regional Council,"165,900\n"
7,Taranaki Regional Council,"119,600\n"
8,Horizons Regional Council,"234,500\n"
9,Greater Wellington Regional Council,"521,500\n"


Some data cleansing to make the table usable.

In [40]:
#Reindex - to drop the first row
df_pop=df_pop.reindex(df_pop.index.drop(0))

#Replace the new line character and comma
df_pop=df_pop.replace(['\\n',','],'', regex=True) 

#Get rid of the redundant words
df_pop=df_pop.replace(['Regional Council', 'District Council','City Council', 'Council'],'', regex=True) 

#Manually purge some values
df_pop=df_pop.replace(['Horizons', 'Greater Wellington'],['Manawatu-Wanganui','Wellington'], regex=True) 

In [45]:
df_pop

,Region,Population
1,Northland,179100
2,Auckland,1695900
3,Waikato,468800
4,Bay of Plenty,305700
5,Gisborne,49100
6,Hawke's Bay,165900
7,Taranaki,119600
8,Manawatu-Wanganui,234500
9,Wellington,521500
10,Tasman,52100


In [42]:
df_pop.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16 entries, 1 to 16
Data columns (total 2 columns):
Region        16 non-null object
Population    16 non-null object
dtypes: object(2)
memory usage: 384.0+ bytes


The `"Population"` column here is stored as string. To utilise this field going forward, it's best to convert this to number (float).

In [46]:
df_pop['Population'] = df_pop['Population'].astype(float)
df_pop.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16 entries, 1 to 16
Data columns (total 2 columns):
Region        16 non-null object
Population    16 non-null float64
dtypes: float64(1), object(1)
memory usage: 384.0+ bytes


Finally, the population table is ready to be used.

## Income from StatsNZ
[Back to top](#Retrieving-NZ-Demographics-Information)

StatsNZ offers a comprehensive collection of interactive tables spanning across a wide range of topics. One can locate a theme (e.g. income) and find a base table easily that contains the information required. Once you find an optimal base table, you can customise the layout and export the table as CSV.

If the base table already contains what you need, you can also extract the information using web scraping - the customised layout unfortunately won't work in this regard. Here, we will use the household income history table as an example (StatsNZ, n.d.).

In [83]:
url2 = "http://nzdotstat.stats.govt.nz/wbos/Index.aspx?DataSetCode=TABLECODE7480#"
response2 = requests.get(url2)
print response2
scraping_stat = BeautifulSoup(response2.content, "lxml")

<Response [200]>


In [88]:
stat_table=scraping_stat.findAll("table", attrs={"class" : "DataTable"})
df_stat = pd.read_html(str(stat_table))
df_stat[0]

,\xa0\xa0\xa0\xa0\xa0,Region,Total Regions,Household Type,Total Household Type,Measure,Average Wage and Salary Income,Average Self-employment Income,Average Government Transfer Income,Average Income from All Sources collected,...,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29,Unnamed: 30,Unnamed: 31,Unnamed: 32,Unnamed: 33,Unnamed: 34
0,1998,NaN,618,180,111,910,505,0,0,756,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1999,NaN,633,204,107,943,518,0,0,786,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2000,NaN,652,195,103,950,543,0,0,794,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2001,NaN,708,207,98,1012,595,0,0,845,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2002,NaN,738,230,99,1063,620,0,0,891,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,2003,NaN,776,242,96,1110,642,0,0,912,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,2004,NaN,808,252,99,1157,690,0,0,959,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,2005,NaN,882,234,92,1208,760,0,0,1017,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,2006,NaN,937,213,99,1248,800,0,0,1080,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,2007,NaN,1018,222,109,1345,864,0,0,1151,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Here, we've managed to scrape the table and store it in a DataFrame. But because the table was previously in a "pivot" format, when converting to a tabular DataFrame, new columns are added and the column headings are misaligned. So, we would need to compare it with the web source and find out which column contains the information we want and go from there.

In [91]:
#Select only year and median income from all source collected
df_income=df_stat[0][['\\xa0\\xa0\\xa0\\xa0\\xa0','Average Income from All Sources collected']] 
# due to the column headings being misaligned, the 'Average Income from All Sources collected' column 
# actually represent its median equivalent.

#Rename columns
df_income.columns=['Year','Median Income']
df_income 

,Year,Median Income
0,1998,756
1,1999,786
2,2000,794
3,2001,845
4,2002,891
5,2003,912
6,2004,959
7,2005,1017
8,2006,1080
9,2007,1151


In [94]:
df_income.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21 entries, 0 to 20
Data columns (total 2 columns):
Year             21 non-null int64
Median Income    21 non-null int64
dtypes: int64(2)
memory usage: 408.0 bytes


Both columns are stored as integer. This table can be used as is, or further preped for other purposes.

## Reference
[Back to top](#Retrieving-NZ-Demographics-Information)

Regions of New Zealand. (n.d). In _Wikipedia_. Retrieved June 14, 2019 from https://en.wikipedia.org/wiki/Regions_of_New_Zealand. 

StatsNZ API. (n.d.). _TABLECODE8135_. Retrieved June 10, 2017, from https://statisticsnz.portal.azure-api.net/docs/services/58c6163976be431aac417590/operations/58c6163c76be4306d83942bd

StatsNZ. (n.d.). _TABLECODE7480_. Retrieved June 14, 2019, from http://nzdotstat.stats.govt.nz/wbos/Index.aspx?DataSetCode=TABLECODE7480#

## Appendix
[Back to top](#Retrieving-NZ-Demographics-Information)

In 2017 to 2018, some demographics data can be obtained from the API provided by StatsNZ (StatsNZ API, n.d.). This was a temporal experiment, with limited documentations. In the initial exploration stage, it was discovered that a lot of the features provided by the API were not fully functional. Therefore, those features have not been applied here for this exercise. And if you run the below code from Dec 2018 onward, it will fail. But below is how it would have worked.
<br/>
<br/>

By querying the catalogue on the API's web interface, we can locate a target endpoint table - TABLECODE8135. This table is for "Total household income (grouped) by household composition, for households in occupied private dwellings, 2006 and 2013 Censuses (RC, TA, AU)".

Here is the outline of the API as provided in the documentation:

__Request URL:__

`https://statisticsnz.azure-api.net/nzdotstat/v1.0/odata/TABLECODE8135[?$expand][&$filter][&$select][&$orderby][&$top][&$skip][&$count]`

__Request parameters:__

 * `$expand` (optional) string - Expands related entities inline.
 * `$filter` (optional) string - Filters the results, based on a Boolean condition.
 * `$select` (optional) string - Selects which properties to include in the response.
 * `$orderby` (optional) string - Sorts the results.
 * `$top` (optional) integer - Format - int32. Returns only the first n results.
 * `$skip` (optional) integer - Format - int32. Skips the first n results.
 * `$count` (optional) boolean - Includes a count of the matching results in the response.

__Request headers:__
* `Ocp-Apim-Subscription-Key`
<br>
<br>

Here, all the records in income table will be queried with the below URL and subscription Key; filters will be applied afterwards.

In [109]:
#Send request to API
url = "https://statisticsnz.azure-api.net/nzdotstat/v1.0/odata/TABLECODE8135"
headers = { 'Ocp-Apim-Subscription-Key': '0c08deadc8b94c9f8bba0a44702f1053' }
income = requests.get(url, headers=headers)
#or income = requests.get(url, headers=headers, proxies=massey_proxies) if running from Massey's domain
print income

<Response [200]>


A successful response is received. We will first display the response in Json format to get an understanding of the response structure.

In [110]:
#Call PrettyPrinter from the pprint library, set indentation to 2
pp = pprint.PrettyPrinter(indent=2) 

#Call method pprint and apply to Json format
pp.pprint(income.json())

{ u'@odata.context': u'https://statisticsnz.azure-api.net/nzdotstat/v1.0/odata/$metadata#TABLECODE8135',
  u'@odata.nextLink': u'https://statisticsnz.azure-api.net/nzdotstat/v1.0/odata/TABLECODE8135/?$skiptoken=+RID:SwMlAKIMlBspEQMAAAAAAA==#RT:1#TRC:1000#FPC:ASkRAwAAAAAAkNADAAAAAAA=',
  u'value': [ { u'Entity': u'Household',
                u'Group': u'People',
                u'IDAREA_Code': u'001',
                u'IDAREA_Name': u'Far North District',
                u'IDHOUSEHOLDCOMP_Code': u'1',
                u'IDHOUSEHOLDCOMP_Name': u'One-family household (with or without other people)',
                u'IDINCOME_Code': u'1',
                u'IDINCOME_Name': u'$20,000 or less',
                u'IDYEAR_Code': u'2006',
                u'IDYEAR_Name': u'2006',
                u'Value': 1275,
                u'tablecode': u'TABLECODE8135'},
              { u'Entity': u'Household',
                u'Group': u'People',
                u'IDAREA_Code': u'001',
                u'IDAR

The response contains three top-level items:
* `@odata.context` appears to hold a URL for metadata information for this endpoint and it is not relevant at this point.
* `@odata.nextLink` seems to contain the URL of the next batch of information. It was discovered in the exploration stage that the API will only return 1000 rows at a time.
* `value` contains the actual Census income data.

Next, the results are loaded into a Pandas DataFrame.

In [111]:
#Extract only the "value" item from the json string
df_income = pd.DataFrame(income.json()['value'])
df_income.info()
df_income.head(5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 12 columns):
Entity                  1000 non-null object
Group                   1000 non-null object
IDAREA_Code             1000 non-null object
IDAREA_Name             1000 non-null object
IDHOUSEHOLDCOMP_Code    1000 non-null object
IDHOUSEHOLDCOMP_Name    1000 non-null object
IDINCOME_Code           1000 non-null object
IDINCOME_Name           1000 non-null object
IDYEAR_Code             1000 non-null object
IDYEAR_Name             1000 non-null object
Value                   995 non-null float64
tablecode               1000 non-null object
dtypes: float64(1), object(11)
memory usage: 93.8+ KB


,Entity,Group,IDAREA_Code,IDAREA_Name,IDHOUSEHOLDCOMP_Code,IDHOUSEHOLDCOMP_Name,IDINCOME_Code,IDINCOME_Name,IDYEAR_Code,IDYEAR_Name,Value,tablecode
0,Household,People,001,Far North District,1,One-family household (with or without other pe...,1,"$20,000 or less",2006,2006,1275.00,TABLECODE8135
1,Household,People,001,Far North District,1,One-family household (with or without other pe...,1,"$20,000 or less",2013,2013,1089.00,TABLECODE8135
2,Household,People,001,Far North District,1,One-family household (with or without other pe...,2,"$20,001 - $30,000",2006,2006,1911.00,TABLECODE8135
3,Household,People,001,Far North District,1,One-family household (with or without other pe...,2,"$20,001 - $30,000",2013,2013,1386.00,TABLECODE8135
4,Household,People,001,Far North District,1,One-family household (with or without other pe...,3,"$30,001 - $50,000",2006,2006,2646.00,TABLECODE8135


1000 rows of data have been received from the response, with 12 columns. The above results provide an outline of what the table looks like. Since the API will only return 1000 rows of entries at a time, we will construct a function to automate the retrieval process.

In [112]:
#This UDF will fetch 1000 rows and return it as as df along with the URL for the next page
def fetch_one_page(url):
    
    headers = { 'Ocp-Apim-Subscription-Key': '0c08deadc8b94c9f8bba0a44702f1053' }
    response = requests.get(url, headers=headers)
    # or response = requests.get(url, headers=headers, proxies=massey_proxies) if running from Massey's domain

    if not response.ok: # if response.status_code is >= 400 (unsuccessful), raise an exception
        raise Exception, 'Unsuccessful request; status code: {0} {1} {2}'.format(response.status_code, 
                                                                                 response.reason, response.text)

    json = response.json() # parse the response string as json format
    
    df_page = pd.DataFrame(json['value']) # extract the "value" item and load into DataFrame

    if '@odata.nextLink' in json: # if there's nextLink, means there're more pages to load
        next_url = json['@odata.nextLink']
    else:
        next_url = None
    
    return (df_page, next_url) # return a tuple with the DataFrame and the URL for the next page

In [114]:
#This UDF will retrieve all data and return the final DataFrame
def fetch_all():
    url = "https://statisticsnz.azure-api.net/nzdotstat/v1.0/odata/TABLECODE8135"

    df_result = pd.DataFrame() # start with an empty DataFrame

    request_num = 0 # start the counter
    while url is not None: # iterate this until there's no next page
        print("Requesting page {}...".format(request_num)) 
        df_page, url = fetch_one_page(url) # unpack the tuple into 2 objects, next_url will be passed as new url
        df_result = df_result.append(df_page) # append the DataFrame to the final DataFrame
        request_num += 1
    return df_result

In [115]:
df_income = fetch_all()
print(df_income.info())

Requesting page 0...
Requesting page 1...
Requesting page 2...
Requesting page 3...
Requesting page 4...
Requesting page 5...
Requesting page 6...
Requesting page 7...
Requesting page 8...
Requesting page 9...
Requesting page 10...
Requesting page 11...
Requesting page 12...
Requesting page 13...
Requesting page 14...
Requesting page 15...
Requesting page 16...
Requesting page 17...
Requesting page 18...
Requesting page 19...
Requesting page 20...
Requesting page 21...
Requesting page 22...
Requesting page 23...
Requesting page 24...
Requesting page 25...
Requesting page 26...
Requesting page 27...
Requesting page 28...
Requesting page 29...
Requesting page 30...
Requesting page 31...
Requesting page 32...
Requesting page 33...
Requesting page 34...
Requesting page 35...
Requesting page 36...
Requesting page 37...
Requesting page 38...
Requesting page 39...
Requesting page 40...
Requesting page 41...
Requesting page 42...
Requesting page 43...
Requesting page 44...
Requesting page 45..

The full DataFrame is returned with 50000 entries. This is because the table contains various income-related items for each income group, household composition and district/region. By interacting with API's web interface, it was discovered that the information we are after (median income) is one of the items returned in the `"IDINCOME_Name"` column. This item is also displayed for each household composition types (as in `"IDHOUSEHOLDCOMP_Name"`). Therefore, to filter for the information of interest, we would need to find out the code for the median income under `"IDINCOME_Name"` and the code for the overall household composition type under `"IDHOUSEHOLDCOMP_Name"`. The codes are displayed using the below command. 

In [46]:
#Join two columns into one 
df_income['Income_Code']= df_income[['IDINCOME_Name', 'IDINCOME_Code']].apply(lambda x: ' '.join(x), axis=1)

#Show unique entries
np.unique(df_income['Income_Code'])

array([u'$100,001 or more 6', u'$20,000 or less 1', u'$20,001 - $30,000 2',
       u'$30,001 - $50,000 3', u'$50,001 - $70,000 4',
       u'$70,001 - $100,000 5', u'Median household income 88',
       u'Not stated 9', u'Total households stated 99',
       u'Total households, grouped total household income 999'], dtype=object)

In [47]:
#Join two columns into one 
df_income['Household_Code']= df_income[['IDHOUSEHOLDCOMP_Name','IDHOUSEHOLDCOMP_Code']].apply(lambda x: ' '.join(x), axis=1)

#Show unique entries
np.unique(df_income['Household_Code'])

array([u'Household composition unidentifiable 6',
       u'One-family household (with or without other people) 1',
       u'One-person household 5', u'Other multi-person household 4',
       u'Three or more family household (with or without other people) 3',
       u'Total households stated 9',
       u'Total households, household composition 99',
       u'Two-family household (with or without other people) 2'], dtype=object)

From the above results we can see that the income item we need is `"Median household income"` with a code of 88; the overall household composition type is `"Total households, household composition"` with a code of 99. Based on this, we will filter the table to contain the median income information for total household in 2013.

In [48]:
df_income= df_income[(df_income['IDINCOME_Code'] == '88') & (df_income['IDYEAR_Name'] =='2013') 
                     &(df_income['IDHOUSEHOLDCOMP_Code'] == '99')]
df_income.head()

,Entity,Group,IDAREA_Code,IDAREA_Name,IDHOUSEHOLDCOMP_Code,IDHOUSEHOLDCOMP_Name,IDINCOME_Code,IDINCOME_Name,IDYEAR_Code,IDYEAR_Name,Value,tablecode,Income_Code,Household_Code
153,Household,People,001,Far North District,99,"Total households, household composition",88,Median household income,2013,2013,42100.0,TABLECODE8135,Median household income 88,"Total households, household composition 99"
313,Household,People,002,Whangarei District,99,"Total households, household composition",88,Median household income,2013,2013,52000.0,TABLECODE8135,Median household income 88,"Total households, household composition 99"
473,Household,People,003,Kaipara District,99,"Total households, household composition",88,Median household income,2013,2013,42500.0,TABLECODE8135,Median household income 88,"Total households, household composition 99"
633,Household,People,01,Northland Region,99,"Total households, household composition",88,Median household income,2013,2013,47000.0,TABLECODE8135,Median household income 88,"Total households, household composition 99"
793,Household,People,011,Thames-Coromandel District,99,"Total households, household composition",88,Median household income,2013,2013,41400.0,TABLECODE8135,Median household income 88,"Total households, household composition 99"


It is also noticed that the districts under `"IDAREA_Name"` are coded with a 3-digit number (as in `"IDAREA_Code"`) while the regions are coded with a 2-digit number. Based on this, we can filter the table further to contain only regional related median income values.

In [49]:
#Filter for records with 2-digit area code
df_income=df_income[df_income['IDAREA_Code'].str.len() == 2].reset_index()
df_income

,index,Entity,Group,IDAREA_Code,IDAREA_Name,IDHOUSEHOLDCOMP_Code,IDHOUSEHOLDCOMP_Name,IDINCOME_Code,IDINCOME_Name,IDYEAR_Code,IDYEAR_Name,Value,tablecode,Income_Code,Household_Code
0,633,Household,People,01,Northland Region,99,"Total households, household composition",88,Median household income,2013,2013,47000.0,TABLECODE8135,Median household income 88,"Total households, household composition 99"
1,626,Household,People,02,Auckland Region,99,"Total households, household composition",88,Median household income,2013,2013,76500.0,TABLECODE8135,Median household income 88,"Total households, household composition 99"
2,833,Not defined,Not defined,03,Waikato Region,99,"Total households, household composition",88,Median household income,2013,2013,59600.0,TABLECODE8135,Median household income 88,"Total households, household composition 99"
3,593,Not defined,Not defined,04,Bay of Plenty Region,99,"Total households, household composition",88,Median household income,2013,2013,54600.0,TABLECODE8135,Median household income 88,"Total households, household composition 99"
4,353,Not defined,Not defined,05,Gisborne Region,99,"Total households, household composition",88,Median household income,2013,2013,50500.0,TABLECODE8135,Median household income 88,"Total households, household composition 99"
5,113,Not defined,Not defined,06,Hawke's Bay Region,99,"Total households, household composition",88,Median household income,2013,2013,53200.0,TABLECODE8135,Median household income 88,"Total households, household composition 99"
6,713,Not defined,Not defined,07,Taranaki Region,99,"Total households, household composition",88,Median household income,2013,2013,58400.0,TABLECODE8135,Median household income 88,"Total households, household composition 99"
7,993,Not defined,Not defined,08,Manawatu-Wanganui Region,99,"Total households, household composition",88,Median household income,2013,2013,50000.0,TABLECODE8135,Median household income 88,"Total households, household composition 99"
8,153,Not defined,Not defined,09,Wellington Region,99,"Total households, household composition",88,Median household income,2013,2013,74300.0,TABLECODE8135,Median household income 88,"Total households, household composition 99"
9,313,Not defined,Not defined,12,West Coast Region,99,"Total households, household composition",88,Median household income,2013,2013,55000.0,TABLECODE8135,Median household income 88,"Total households, household composition 99"


Finally, we will further filter the table to only contain the columns of interest - Region and Median Income. These columns will be renamed and the word "Region" will be removed from the Region entries.

In [50]:
df_income=df_income[['IDAREA_Name','Value']] # select only two columns
df_income.columns=['Region','MedianIncome'] # rename columns

# find pattern that is space+"Region"
reg = re.compile(' Region') 

# UDF to remove the word "Region"
def remove_reg(x): 
    return re.sub(reg, '', x) #Substitute "region" with nothing

# apply UDF
df_income['Region'] = df_income['Region'].apply(remove_reg)

df_income

c:\python27\lib\site-packages\ipykernel\__main__.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,Region,MedianIncome
0,Northland,47000.0
1,Auckland,76500.0
2,Waikato,59600.0
3,Bay of Plenty,54600.0
4,Gisborne,50500.0
5,Hawke's Bay,53200.0
6,Taranaki,58400.0
7,Manawatu-Wanganui,50000.0
8,Wellington,74300.0
9,West Coast,55000.0
